In [ ]:
!pip install noisereduce
!pip install gdown librosa keras scikit-learn

In [30]:
import zipfile
import os
import numpy as np
import librosa
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
import pickle

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [31]:
# Extract the dataset
zip_path = '/content/drive/MyDrive/TASKS/ravdess.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('ravdess')
dataset_dir = 'ravdess'

In [32]:
def LoadAndExtract(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

In [33]:
# Get all file paths and corresponding labels
file_paths = []
labels = []

# Define a mapping from emotion numbers to labels
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Parse file paths and extract labels
for dirpath, dirnames, filenames in os.walk(dataset_dir):
    for filename in filenames:
        if filename.endswith('.wav'):
            file_paths.append(os.path.join(dirpath, filename))
            emotion = filename.split('-')[2]
            labels.append(emotion_map[emotion])

In [34]:
# Convert labels to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Save the label encoder to Google Drive
label_encoder_path = '/content/drive/MyDrive/TASKS/EmotionRecognitionModel.pkl'
with open(label_encoder_path, 'wb') as f:
    pickle.dump(le, f)

# Load and extract features from all audio files
data = np.array([LoadAndExtract(fp) for fp in file_paths])

In [35]:
# Convert labels to one-hot encoding
labels = keras.utils.to_categorical(labels, num_classes=len(le.classes_))

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (data.shape[1], 1)

# Reshape data for Conv1D
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [36]:
# Build the model
model = Sequential()
model.add(Conv1D(128, 5, padding='same', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(len(le.classes_), activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test))

Epoch 1/50
72/72 [==============================] - 1s 8ms/step - loss: 11.5531 - accuracy: 0.1667 - val_loss: 4.0994 - val_accuracy: 0.2222
Epoch 2/50
72/72 [==============================] - 0s 5ms/step - loss: 8.2513 - accuracy: 0.1801 - val_loss: 2.9756 - val_accuracy: 0.2535
Epoch 3/50
72/72 [==============================] - 0s 4ms/step - loss: 6.1979 - accuracy: 0.2023 - val_loss: 3.3168 - val_accuracy: 0.2413
Epoch 4/50
72/72 [==============================] - 0s 5ms/step - loss: 4.6949 - accuracy: 0.2127 - val_loss: 2.3251 - val_accuracy: 0.3021
Epoch 5/50
72/72 [==============================] - 0s 6ms/step - loss: 3.6408 - accuracy: 0.2422 - val_loss: 2.0067 - val_accuracy: 0.2812
Epoch 6/50
72/72 [==============================] - 0s 6ms/step - loss: 2.9571 - accuracy: 0.2661 - val_loss: 1.9660 - val_accuracy: 0.3073
Epoch 7/50
72/72 [==============================] - 0s 5ms/step - loss: 2.4168 - accuracy: 0.2908 - val_loss: 1.7981 - val_accuracy: 0.3264
Epoch 8/50
72/72 [=

In [37]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.5243055820465088


In [38]:
model.save('/content/drive/MyDrive/TASKS/EmotionRecognitionModel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
